In [3]:
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from PyRadioLoc.Utils.GeoUtils import GeoUtils
from sklearn.ensemble import RandomForestRegressor
%run Imports.ipynb

def write_to_csv(x_pred, y_pred, file_path):
    #labels = ["rssi_1_1", "rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"]
    #y_pred_dataframe = pd.DataFrame(y_pred)
    res = pd.concat([x_pred, y_pred], axis=1)
    res.to_csv(path_or_buf=file_path, index = False)

In [72]:
db_train = pd.read_csv("databases/Outdoor_X_train_90%.csv")
X_train = db_train.drop("idx", axis = 1)
X_train = X_train.drop("lat", axis = 1)
X_train = X_train.drop("lon", axis = 1)
#lon_train = db_train["lon"]
#lat_train = db_train["lat"]
#X_train = pd.concat([lat_train, lon_train], axis=1)
X_train.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delta_1_12,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,-91.801429,-92.775714,-93.930000,-96.246667,-92.3700,-86.497143,-91.915000,-83.340000,-95.8575,0.974286,...,111.046648,-0.359128,0.933288,-2.598763,7,3,4,4,3,-1
1,-87.827143,-90.397143,-88.951429,-67.090000,-82.1725,-81.895714,-86.488333,-92.622000,-97.6000,2.570000,...,71.008261,0.325432,0.945566,2.905572,4,2,5,2,-1,-3
2,-96.500000,-89.047013,-94.131524,-95.600000,-94.3100,-74.561071,-88.900000,-91.629273,-95.8575,-7.452987,...,110.132440,-0.344191,0.938900,-2.727842,8,3,5,5,3,-2
3,-76.247143,-88.987143,-77.557500,-86.900000,-84.5200,-91.645000,-80.279444,-102.828000,-97.8275,12.740000,...,26.342135,0.896160,0.443730,0.495146,2,5,4,-3,-2,1
4,-75.950000,-88.482857,-74.288571,-86.510000,-91.5200,-93.597143,-84.063333,-104.362000,-81.3025,12.532857,...,-15.625616,0.963042,-0.269350,-0.279687,4,7,3,-3,1,4


In [71]:
# Essa Linha y_train = pd.read_csv("databases/Outdoor_y_train_90%.csv")
#y_train = db_train
#y_train = y_train.drop("idx", axis = 1)
#y_train = y_train.drop("lat", axis = 1)
#y_train = y_train.drop("lon", axis = 1)
# Essa linha y_train.head()
y_train_lon = pd.read_csv("Grid/grid_30.csv")
y_train_lon = y_train_lon.drop("lat", axis = 1)
y_train_lon.head()

,lon
0,-34.959850
1,-34.959549
2,-34.959248
3,-34.958948
4,-34.958647


In [55]:
# Essa linha X_test = pd.read_csv("Grid/grid_20.csv")
# Essa linha X_test.head()
X_test = pd.read_csv("databases/Outdoor_X_test_10%.csv")
X_test = X_test.drop("idx", axis = 1)
X_test = X_test.drop("lat", axis = 1)
X_test = X_test.drop("lon", axis = 1)
X_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delta_1_12,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,2.514286,...,94.650653,-0.081080,0.996708,-12.292874,6,5,1,1,5,4
1,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,-18.511429,...,52.781288,0.604859,0.796332,1.316558,3,3,6,0,-3,-3
2,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,-8.922857,...,65.551640,0.413873,0.910335,2.199551,4,2,6,2,-2,-4
3,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,-2.180476,...,81.758935,0.143338,0.989674,6.904462,5,2,5,3,0,-3
4,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,0.004286,...,72.359742,0.303040,0.952978,3.144731,5,2,5,3,0,-3


In [112]:
#reg_cso = RandomForestRegressor(n_estimators = 115, random_state = 42, criterion = 'mae')
#reg_cso.fit(X_train, y_train)
#y_pred_cso = reg_cso.predict(X_test)

# LightGBM 

In [55]:
# Splitting the dataset into the Training set and Test set
#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [56]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [57]:
import lightgbm as lgb

In [58]:
d_train_lon = lgb.Dataset(X_train, label=y_train_lon)
d_train_lon

In [117]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['num_leaves'] = 28
#params['min_data'] = 50
#params['max_depth'] = 10
clf = lgb.train(params, d_train_lon, 10000)

In [118]:
#Prediction
y_pred_lon=clf.predict(X_test)
#y_pred_lon

In [119]:
X_test = pd.read_csv("databases/Outdoor_X_test_10%.csv")
X_test = X_test.drop("lat", axis = 1)
X_test = X_test.drop("lon", axis = 1)

In [120]:
y_pred_lon = pd.DataFrame({'lon': y_pred_lon})
#y_pred_lon

In [121]:
write_to_csv(X_test, y_pred_lon, "fingerprint_LightGBM_grid_20.csv")

In [124]:
parcialResult = pd.concat([X_test, y_pred_lon], axis=1)
parcialResult.head()

,idx,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,...,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lon
0,7080,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,...,-0.081080,0.996708,-12.292874,6,5,1,1,5,4,-34.950584
1,9438,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,...,0.604859,0.796332,1.316558,3,3,6,0,-3,-3,-34.951302
2,8840,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,...,0.413873,0.910335,2.199551,4,2,6,2,-2,-4,-34.950420
3,7629,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,...,0.143338,0.989674,6.904462,5,2,5,3,0,-3,-34.951460
4,7988,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,...,0.303040,0.952978,3.144731,5,2,5,3,0,-3,-34.949914


In [128]:
# Essa Linha y_train = pd.read_csv("databases/Outdoor_y_train_90%.csv")
#y_train = db_train
#y_train = y_train.drop("idx", axis = 1)
#y_train = y_train.drop("lat", axis = 1)
#y_train = y_train.drop("lon", axis = 1)
# Essa linha y_train.head()
y_train_lat = pd.read_csv("Grid/grid_30.csv")
y_train_lat = y_train_lat.drop("lon", axis = 1)
y_train_lat.head()

,lat
0,-8.05955
1,-8.05955
2,-8.05955
3,-8.05955
4,-8.05955


In [129]:
# Essa linha X_test = pd.read_csv("Grid/grid_20.csv")
# Essa linha X_test.head()
X_test = pd.read_csv("databases/Outdoor_X_test_10%.csv")
X_test = X_test.drop("idx", axis = 1)
X_test = X_test.drop("lat", axis = 1)
X_test = X_test.drop("lon", axis = 1)
X_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delta_1_12,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,2.514286,...,94.650653,-0.081080,0.996708,-12.292874,6,5,1,1,5,4
1,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,-18.511429,...,52.781288,0.604859,0.796332,1.316558,3,3,6,0,-3,-3
2,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,-8.922857,...,65.551640,0.413873,0.910335,2.199551,4,2,6,2,-2,-4
3,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,-2.180476,...,81.758935,0.143338,0.989674,6.904462,5,2,5,3,0,-3
4,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,0.004286,...,72.359742,0.303040,0.952978,3.144731,5,2,5,3,0,-3


In [130]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [131]:
import lightgbm as lgb

In [132]:
d_train_lat = lgb.Dataset(X_train, label=y_train_lat)
d_train_lat

In [133]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['num_leaves'] = 28
#params['min_data'] = 50
#params['max_depth'] = 10
clf = lgb.train(params, d_train_lat, 10000)

In [139]:
#Prediction
y_pred_lat=clf.predict(X_test)
#y_pred_lat
y_pred_lat = pd.DataFrame({'lat': y_pred_lat})
y_pred_lat.head()

,lat
0,-8.050560
1,-8.053089
2,-8.054150
3,-8.056325
4,-8.054176


In [140]:
write_to_csv(parcialResult, y_pred_lat, "fingerprint_LightGBM_grid_20.csv")

In [4]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.utils import shuffle

In [5]:
X = X_train
#X.head()
#X = dataset.iloc[:, [0, 12]]
#y = dataset.iloc[:, 
X.head()

,dist_1,dist_2,dist_3,ang_1,cos_1,sin_1,tg_1,ang_2,cos_2,sin_2,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,1.623132,0.504054,1.266725,158.235526,-0.928716,0.370792,-0.399252,-155.889424,-0.912759,-0.408499,...,98.688231,-0.151058,0.988525,-6.544019,7,2,5,5,2,-3
1,0.946625,0.892100,0.682786,-175.257624,-0.996577,-0.082676,0.082960,-83.308432,0.116525,-0.993188,...,56.916564,0.545860,0.837877,1.534967,4,4,3,0,1,1
2,0.813049,1.083588,0.677145,-162.727849,-0.954905,-0.296911,0.310932,-75.519693,0.250047,-0.968234,...,37.152301,0.797033,0.603936,0.757730,3,5,3,-2,0,2
3,0.827905,0.499070,1.382476,138.771771,-0.752090,0.659060,-0.876304,-31.683823,0.850959,-0.525231,...,59.895457,0.501579,0.865112,1.724775,4,2,6,2,-2,-4
4,0.702384,0.731350,1.042135,165.550330,-0.968367,0.249529,-0.257681,-59.864250,0.502050,-0.864838,...,52.395419,0.610209,0.792241,1.298312,3,3,4,0,-1,-1


In [6]:
y = y_train_rssi_1_1
y.head()

,rssi_1_1
0,-87.185714
1,-85.330000
2,-90.572857
3,-96.702857
4,-91.864286


In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [8]:
seed = 42
X, y = shuffle(X, y, random_state=seed)

In [9]:
#test = list(map(pot_2,np.arange(-2,10, dtype=float) ))
#test 

In [12]:
def svr_model(X, y):
    gsc = GridSearchCV(
        estimator=SVR(),
        param_grid={
            'C': list(map(pot_2,np.arange(-2,14, dtype=float) )), #[0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256],
            'epsilon': [0.01, 0.05, 0.1, 0.5, 1, 2, 4],
            'gamma': list(map(pot_2,np.arange(-5,4, dtype=float) )), #[0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8],
            'kernel': ['rbf'] # ['sigmoid', 'linear',]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, y.values.ravel())
    best_params = grid_result.best_params_
    best_svr = SVR(kernel=best_params["kernel"], C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)

    scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error'}

    scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
    return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean())), "| best_svr :", best_svr

In [13]:
print(svr_model(X,y))

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

('MAE :', 2.032841652002269, '| RMSE :', 3.270056503797917, '| best_svr :', SVR(C=64.0, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=8.0,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))


In [ ]:
#27/09/20 19:48PM
('MAE :', 2.032841652002269, '| RMSE :', 3.270056503797917, '| best_svr :', SVR(C=64.0, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=8.0,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))

In [ ]:
#27/09/20
('MAE :', 1.9664204614552834, '| RMSE :', 3.227352843669652, '| best_svr :', SVR(C=32.0, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=16.0,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))

In [ ]:
('MAE :', 2.083546343441733, '| RMSE :', 3.3166580120758025, '| best_svr :', SVR(C=100, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=5,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))

In [ ]:
('MAE :', 2.032766183511892, '| RMSE :', 3.270037850352753, '| best_svr :', SVR(C=64, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=8,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))

# Grid Search LightGBM:

In [15]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.utils import shuffle
import lightgbm as lgb
%run Imports.ipynb

In [16]:
X_LightGBM = X_train

In [17]:
y_LightGBM = y_train_rssi_1_1

In [18]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_LightGBM = scaler.fit_transform(X_LightGBM)

In [19]:
seed = 42
X_LightGBM, y_LightGBM = shuffle(X_LightGBM, y_LightGBM, random_state=seed)

# Step 1

In [54]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

model = lgb.LGBMRegressor( 
    boosting_type="goss",
    n_estimators=50, 
    max_depth=8,
    feature_fraction=0.7,  
    learning_rate=0.1,    
)

params_opt = {'n_estimators':range(200, 1000, 100)}
gridSearchCV = GridSearchCV(estimator = model, 
    param_grid = params_opt, 
    scoring='neg_mean_squared_error',
    n_jobs=4,
    iid=False, 
    verbose=1,
    cv=5)
gridSearchCV.fit(X_train,y_train_rssi_1_1)
gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   17.4s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:   49.4s finished


({'n_estimators': 200}, -1.4892469532523415e+77)

# Step 2

In [55]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

model = lgb.LGBMRegressor( 
    boosting_type="goss",
    n_estimators=200, 
    max_depth=8,
    feature_fraction=0.7,  
    learning_rate=0.1,    
)

params_opt = {'max_depth': range(3, 10, 1), 'num_leaves': range(10, 110, 10)}
gridSearchCV = GridSearchCV(estimator = model, 
    param_grid = params_opt, 
    scoring='neg_mean_squared_error',
    n_jobs=4,
    iid=False, 
    verbose=1,
    cv=5)
gridSearchCV.fit(X_train,y_train_rssi_1_1)
gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  76 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done 376 tasks      | elapsed:   18.7s
[Parallel(n_jobs=4)]: Done 876 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 900 out of 900 | elapsed:  1.1min finished


({'max_depth': 3, 'num_leaves': 5}, -6.8112097852742e+58)

In [50]:
from sklearn.model_selection import GridSearchCV
y_train= y_train_rssi_1_2
y_test= y_train_rssi_1_2
params_test1={'max_depth': np.arange(3,12,1), 'num_leaves': np.arange(5,150,5)}
              
gsearch1 = GridSearchCV(estimator = lgb.LGBMRegressor(boosting_type='goss',metrics='mse',learning_rate=0.1, n_estimators=100, max_depth=8,feature_fraction = 0.8), 
                       param_grid = params_test1, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
gsearch1.fit(X_train,y_train)
gsearch1.best_params_, gsearch1.best_score_

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'max_depth': 10, 'num_leaves': 50}, -3.634671762602775e+17)

# Step 3

In [35]:
params_test2={'max_bin': np.arange(5,256,10), 'min_data_in_leaf': np.arange(1,102,10)}
              
gsearch2 = GridSearchCV(estimator = lgb.LGBMRegressor(boosting_type='goss',metrics='mse',learning_rate=0.1, n_estimators=100, max_depth=6, num_leaves=10, feature_fraction = 0.8), 
                       param_grid = params_test2, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
gsearch2.fit(X_train,y_train)
gsearch2.best_params_, gsearch2.best_score_

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'max_bin': 5, 'min_data_in_leaf': 61}, -1.0915389708511447e+23)

# Step 4

In [40]:
params_test3={'feature_fraction': [0.6,0.7,0.8,0.9,1.0]}
y_train= y_train_rssi_3_3            
gsearch3 = GridSearchCV(estimator = lgb.LGBMRegressor(boosting_type='goss',metrics='mse',learning_rate=0.1, n_estimators=1000, max_depth=6, num_leaves=10, feature_fraction = 0.8, max_bin = 5, min_data_in_leaf = 61), 
                       param_grid = params_test3, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_, gsearch3.best_score_

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'feature_fraction': 0.6}, -8.741721636386762e+34)

# Step 5

In [42]:
params_test4={'lambda_l1': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0,2.0, 4.0, 6.0, 8.0, 10.0],
              'lambda_l2': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0,2.0, 4.0, 6.0, 8.0, 10.0]
}
              
gsearch4 = GridSearchCV(estimator = lgb.LGBMRegressor(boosting_type='goss',metrics='mse',learning_rate=0.1, n_estimators=1000, max_depth=6, num_leaves=10, feature_fraction = 0.6, max_bin = 5, min_data_in_leaf = 61), 
                       param_grid = params_test4, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
gsearch4.fit(X_train,y_train)
gsearch4.best_params_, gsearch4.best_score_

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'lambda_l1': 8.0, 'lambda_l2': 10.0}, -45.335813926708084)

# Step 6

In [47]:
params_test5={'min_split_gain':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0, 4.0, 6.0, 8.0, 10.0]}
y_train= y_train_rssi_1_1          
gsearch5 = GridSearchCV(estimator = lgb.LGBMRegressor(boosting_type='goss',metrics='mse',learning_rate=0.1, n_estimators=1000, max_depth=6, num_leaves=10, feature_fraction = 0.6, max_bin = 5, min_data_in_leaf = 61, lambda_l1 = 8, lambda_l2 = 10), 
                       param_grid = params_test5, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
gsearch5.fit(X_train,y_train)
gsearch5.best_params_, gsearch5.best_score_

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'min_split_gain': 10.0}, -461.40006304556834)

# Step 7

In [44]:
model=lgb.LGBMRegressor(boosting_type='goss',metrics='mse',learning_rate=0.1, n_estimators=1000, max_depth=6, num_leaves=10, feature_fraction = 0.8, max_bin = 5, min_data_in_leaf = 61, lambda_l1 = 8, lambda_l2 = 10, min_split_gain = 0.1) 
model.fit(X_train,y_train)
y_pre=model.predict(X_test)
print("acc:",metrics.accuracy_score(y_test,y_pre))
print("auc:",metrics.roc_auc_score(y_test,y_pre))

NameError: name 'metrics' is not defined

# GridSearch Original

In [99]:
# Set params
params = {'boosting_type': 'goss',
          'learning_rate': 0.1,
          'metric': 'mse',
          'max_depth': 7,
          'num_leaves': 32,
          'n_estimators': 200,
          'min_data_in_leaf': 21,
          'feature_fraction': 1,
          'lambda_l1': 8.5,
          'lambda_l2': 10,
          'min_split_gain': 10,
          #'min_child_weight': 42,
          
         }
  
# Create parameters to search
gridParams = {
    #'learning_rate': [0.005, 0.1, 0.25,0.5, 1.0],
    #'num_leaves': list(map(pot_2,np.arange(3,10))),#[16,18,20,22,24,26,28,30,32,34,36,38,40,42,54,56,58,60,62,64,66],
    #'max_depth': np.arange(5,11),#[-1,10,20,40,80], #10
    #'bagging_fraction': np.arange(0.25,1.05,0.05),
    #'feature_fraction': np.arange(0.6,1.01,0.1),
    #'max_bin': np.arange(205,255,5),
    #'min_data_in_leaf': np.arange(1,52,10),
    #'lambda_l1': np.arange(8,10.5,0.5),
    #'lambda_l2': np.arange(8,10.5,0.5),
    #'min_split_gain':[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
    #'top_rate': np.arange(0,1.01,0.1),
    #'other_rate': np.arange(0,1.01,0.1),
    'min_child_weight': np.arange(11,42,10), #np.arange(30,43,1),
    
}

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMRegressor(boosting_type= 'goss', silent = True,)
          
# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=0,cv=5)
                         
# Run the grid
grid.fit(X_LightGBM, y_LightGBM.values.ravel())

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

{'min_child_weight': 41}
0.07401638798350865


C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [100]:
test = np.arange(11,42,10)
test

array([11, 21, 31, 41])

In [ ]:
'top_rate': np.arange(0.0,1.01,0.1)
'other_rate': np.arange(0.0,1.01,0.1)
'min_child_weight': np.arange(22,62,10)

In [ ]:
    #'min_split_gain':[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
    #'top_rate': np.arange(0,1.01,0.1),
    #'other_rate': np.arange(0,1.01,0.1),
    'min_child_weight': np.arange(30,43,1),